In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import os
import seaborn
from tqdm import tqdm

trainPath = 'data/Dataset_1_train_asset.csv'
targetPath = 'data/Dataset_1_train_payoff.csv'

trainData = pd.read_csv(trainPath)
trainData.columns = [i for i in range(0, 31)]
target = pd.read_csv(targetPath)

In [ ]:
# exercise 1 - BlackScholes Model

N = norm.cdf

def bs_call(K: float, #underlying asset's price
            T: float, #time till maturity
            S: float = 1.0, # strike price
            r: float = 0.0,  # risk-free
            sigma: float = 0.158):
    
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T) * N(d2), N(-d1)

def bs_put(K: float, #underlying asset's price
           T: float, #time till maturity
           S: float = 1.0, #strike price
           r: float = 0.0, # risk-free
           sigma: float = 0.158):
    
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K*np.exp(-r*T)*N(-d2) - S*N(-d1)

In [ ]:
def calculate_hedge(df: pd.DataFrame):
    deltas = pd.DataFrame(index=range(df.shape[0]),columns=range(df.shape[1]))
    portfolio_value_50 = pd.DataFrame(index=range(df.shape[0]),columns=range(df.shape[1]))
    portfolio_value_99 = pd.DataFrame(index=range(df.shape[0]),columns=range(df.shape[1]))
    
    for i in tqdm(range(0, df.shape[0])):
        short_position = 0.0
        for j in range(0, df.shape[1]):
            S = 100
            initial_price = 100
            K = df.loc[i, :].values[j]
            T = (31 - j)/365
            result, delta = bs_call(K=K, T=T, S=S)
            short_position += -result
            deltas.loc[i, j] = delta
            option_value = max(0, S - K)
            pnl_50 = (option_value + short_position*S) - initial_price*0.5
            pnl_99 = (option_value + short_position*S) - initial_price*0.01
            portfolio_value_50.loc[i, j] = pnl_50
            portfolio_value_99.loc[i, j] = pnl_99
            
    return deltas, portfolio_value_50, portfolio_value_99


In [ ]:
bs_deltas, val50, val99 = calculate_hedge(trainData)

In [ ]:
def plot_figures(deltas: pd.DataFrame, df: pd.DataFrame): 
    fig, axs = plt.subplots(2, 3)
    fig.set_size_inches(18.5, 10.5)
    fig.suptitle('Delta spread at different time intervals', size=45)

    axs[0,0].plot(df.loc[:, 0], deltas.loc[:, 0], 'bo')
    axs[0,0].set_title('Time 0')
    axs[0,1].plot(df.loc[:, 1], deltas.loc[:, 1], 'bo')
    axs[0,1].set_title('Time 1')
    axs[0,2].plot(df.loc[:, 5], deltas.loc[:, 5], 'bo')
    axs[0,2].set_title('Time 5')
    axs[1,0].plot(df.loc[:, 15], deltas.loc[:, 15], 'bo')
    axs[1,0].set_title('Time 15')
    axs[1,1].plot(df.loc[:, 25], deltas.loc[:, 25], 'bo')
    axs[1,1].set_title('Time 25')
    axs[1,2].plot(df.loc[:, 30], deltas.loc[:, 30], 'bo')
    axs[1,2].set_title('Time 30')

    for ax in axs.flat:
        ax.set(xlabel='Spot price', ylabel='Delta')

In [ ]:
plot_figures(bs_deltas, trainData)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Set up device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define stock price path, payoffs, and transaction costs
S_list = torch.Tensor(trainData.values.tolist())
payoffs = torch.Tensor([item for sublist in target.values for item in sublist])
transaction_costs = torch.Tensor([0.0] * 9_999)

# Set hyperparameters
batch_size = 4
num_epochs = 10
learning_rate = 0.0001

# Define neural network model
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x[:, -1, :])
        return x

In [ ]:
model = Net(1, 16, 1).to(device)

# Define loss function
def P_L(delta, S, K, c):
    # calculate (delta · S)T
    delta_S = torch.sum(torch.cat([delta]*31, dim=1).unsqueeze(-1) * S, dim=-1)
    # calculate F(ST) = max(ST - K, 0)
    F_ST = torch.max(S[-1] - K, torch.zeros_like(S[-1]))
    # calculate CT(delta) = c * sum(|delta_k - delta_k-1| * Sk)
    if all(c) == 0:
        P_L = delta_S - F_ST.transpose(0, 1)
    else: ## i didnt feel like fixing the size of CT_delta, will fix later so that it won't fail with c>0
        P_L = delta_S - F_ST.transpose(0, 1) - c
    return P_L

def loss_fn(delta, S, K, c, beta=1):
    P_L_val = P_L(delta, S, K, c)
    # calculate expected shortfall (ES) at 50% and 99% levels
    ES50 = torch.median(P_L_val)
    ES99 = torch.kthvalue(P_L_val, int(0.99 * P_L_val.numel()))[0]
    
    # calculate the risk measure
    risk_measure = 1.0 / (1 + beta) * (ES50 + beta * ES99)
    # calculate the loss as the negative of the P&L
    #loss = -torch.mean(P_L_val * torch.cat([risk_measure.unsqueeze(-1)]*31, dim=-1))
    return risk_measure

# Set up optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Set up dataset and iterator
dataset = torch.utils.data.TensorDataset(S_list, payoffs, transaction_costs)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Training loop
criterion = nn.MSELoss(reduction='mean')
running_loss = 0.0
K = 100
for epoch in range(num_epochs):
    for S, y, c in data_loader:
        S = S.to(device).unsqueeze(-1)
        y = y.to(device)
        c = c.to(device)
        
        # Calculate gradients and update model weights
        optimizer.zero_grad()
        output = model(S)
        pnl = loss_fn(output, S, K, c, beta=1)
        loss = criterion(pnl, y)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()

# Calculate risk measure on validation set
S_val = torch.Tensor([[100, 101, 102, 103, 104],
                     [100, 99, 98, 97, 96],
                     [100, 105, 110, 115, 120]])
y_val = torch.Tensor([4, -4, 20])
c_val = torch.Tensor([0.01, 0.01, 0.01])
S_val = S_val.to(device)
y_val = y_val.to(device)
c_val = c_val.to(device)
losses = -loss_fn(model, S_val, y_val, c_val)
p50 = torch.percentile(losses, 50)
p99 = torch.percentile(losses, 99)
risk_measure = 1 / 2 * (p50 + p99)
print(f'Risk measure: {risk_measure:.4f}')